In [36]:
# Author: Jaehun Kim
# Email: rlawogns1204@unist.ac.kr
# Affiliation: UNIST BME BCILAB
# Date: 2023-06-12
#
# This code implements a tactile information processing model using a spiking
# neural network (SNN). It simulates the processing of tactile information from
# mechanoreceptors in the skin through primary afferent fibers (PA), cuneate nucleus
# neurons (PN and IN), and ultimately, somatosensory cortex neurons. The model
# incorporates lateral inhibition and various receptive field properties to
# represent a realistic processing of touch stimuli.

# with DIGIT-sensor
# pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scipy.io
import time
import numpy as np
from PIL import Image
import io
import datetime
import cv2
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

from Function.snn_IZHIlayers import *
from Function.snn_plot import *
from Function.snn_simulation import *
from Function.snn_stimulation import *
from Function.snn_receptive_field_weights import *

from line_profiler import LineProfiler

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# Check if GPU is available and print the CUDA version
print(f"GPU available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Enable GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the device to 'cpu'
device = 'cpu'
print(f"Using device: {device}")
# Start of the SNN (Spiking Neural Network) code

GPU available: True
CUDA version: 10.1
Using device: cpu


In [ ]:
# Set sensor dimensions (height and width in millimeters)
sensor_h, sensor_w = 19, 16
# Set pixel dimensions (number of pixels in height and width)
# pixel_h, pixel_w = 320, 240///
pixel_h, pixel_w = 64,48
# Set image frames per second (FPS) of tactile sensor
image_FPS = 60

In [35]:
R = ReceptiveFieldWeights(pixel_h, pixel_w, device, type_output = 'single', plot_receptive_field = False)
# R.plot_receptive_field(sa_ind =0, ra_ind =0, no_legend = True)

single
torch.Size([88, 3072])
sa_rf shape: torch.Size([88, 3072]) with height = 11 with width = 8
ra_rf shape: torch.Size([130, 3072]) with height = 13 with width = 10
sa_cn_pn_rf shape:  torch.Size([54, 88]) sa_cn_pn_step_height: 9 sa_cn_pn_step_width: 6
sa_cn_in_rf shape:  torch.Size([54, 88]) sa_cn_in_step_height: 9 sa_cn_in_step_width: 6
ra_cn_pn_rf shape:  torch.Size([88, 130]) ra_cn_pn_step_height: 11 ra_cn_pn_step_width: 8
ra_cn_in_rf shape:  torch.Size([88, 130]) ra_cn_in_step_height: 11 ra_cn_in_step_width: 8
sa_intopn_rf shape:  torch.Size([54, 54])
ra_intopn_rf shape:  torch.Size([88, 88])
cn_pn_sa_rf shape:  torch.Size([28, 54]) cn_pn_sa_rf_step_height: 7 cn_pn_sa_rf_step_width: 4
cn_in_sa_rf shape:  torch.Size([28, 54]) cn_in_sa_rf_step_height: 7 cn_in_sa_rf_step_width: 4
cn_pn_ra_rf shape:  torch.Size([28, 88]) cn_pn_ra_rf_step_height: 7 cn_pn_ra_rf_step_width: 4
cn_in_ra_rf shape:  torch.Size([28, 88]) cn_in_ra_rf_step_height: 7 cn_in_ra_rf_step_width: 4
cn_intopn_rf sha

In [ ]:
# Parameters
num_stim = 1000
pixel_h = 64
pixel_w = 48
F = 0.5
device = 'cpu'  # Or your device of choice

# Generate the stimuli and stack them up
stimulus = torch.zeros((pixel_h, pixel_w, num_stim), device=device)

# Generate stimuli
for freq in range(1, 500, 100):  # Frequency from 1 to 100 with a step of 10
    # Convert the frequency to radians
    frequency_radians = 2 * np.pi * freq / num_stim
    print("Frequency:", freq, "Hz, which is", frequency_radians, "radians.")

    for i in range(num_stim):
        stim_dot = full_stim(F=F, pixel_h=pixel_h, pixel_w=pixel_w)
        
        # Apply sine wave form with the given frequency
        intensity = np.sin(frequency_radians * i)
        stimulus[:, :, i] = stim_dot * intensity  # Apply frequency code to stimulus

    S = SNN(R, device = device, noise_std_val = 0)
    S.feedforward(stimulus)
    mean_firing_rates = plot_SNN(S,layers=['sa','ra','cn'],plot_figure = True)

    plt.figure()
    plt.plot(stimulus[40, 40, :])
    plt.plot(torch.abs(torch.diff(stimulus[40, 40, :])) * 5)
    plt.title("Frequency: " + str(freq) + " Hz")
    plt.show()

    num_neurons = S.ra_spike_times[1].shape[0]

    # Initialize an array to hold the spike counts for each neuron
    spike_counts = np.zeros(num_neurons)

    # Calculate the spike count for each neuron
    for neuron in range(num_neurons):
        spike_counts[neuron] = np.count_nonzero(S.ra_spike_times[1][neuron, :])

    # Plot the spike counts
    plt.figure()
    plt.plot(range(num_neurons), spike_counts)
    plt.title("Spike Count per Neuron")
    plt.xlabel("Neuron Index")
    plt.ylabel("Spike Count")
    plt.show()


In [ ]:
num_trials = 20
num_frequencies = range(1,101, 10)  # Frequencies from 1 to 100 with a step of 10
num_stim = 100
F = 0.5  # Changed from 10 to 0.02 to match the frequency stimulation example
pixel_h = 64
pixel_w = 48
intensity = 20
device = 'cpu'  # Or your device of choice
data = []
labels = []
data_SC = []

# Run the trials
for trial in range(num_trials):
    # Generate the stimuli for each frequency
    for frequency in num_frequencies:
        print(f"Trial: {trial}, Frequency: {frequency} Hz")

        # Generate the stimulus
        stimulus = generate_single_frequency_stimulation(frequency, num_stim=num_stim, pixel_h=pixel_h, pixel_w=pixel_w, F=F, device=device)

        S = SNN(R, device = device, noise_std_val = 0)
        S.feedforward(stimulus)
        mean_firing_rates = plot_SNN(S, plot_figure=False)

        spike_counts = np.zeros(num_neurons)
        for neuron in range(num_neurons):
            spike_counts[neuron] = np.count_nonzero(S.ra_spike_times[1][neuron, :])


        # Save the data and labels
        data.append(mean_firing_rates.cpu().numpy())
        data_SC.append(spike_counts)
        labels.append(frequency)



print(labels)

In [ ]:
# Convert data and labels to numpy arrays

data = np.array(data)
labels = np.array(labels)
print(labels)
# np.save('data_svm_frequency.npy', data)
# np.save('labels_frequency.npy', labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create and train the SVM
clf = svm.SVC(kernel='linear')  # You can change the kernel as needed
clf.fit(X_train, y_train)

# Test the SVM
y_pred = clf.predict(X_test)

# Print the accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test)

misclassified = np.where(y_test != y_pred)

# Print the misclassified labels
print("Misclassified labels: ", y_test[misclassified])

# Print the indices of the misclassified samples
print("Indices of misclassified samples: ", misclassified)

In [ ]:
# Convert data and labels to numpy arrays

data = np.array(data_SC)
labels = np.array(labels)
print(labels)
# np.save('data_svm_frequency.npy', data)
# np.save('labels_frequency.npy', labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create and train the SVM
clf = svm.SVC(kernel='linear')  # You can change the kernel as needed
clf.fit(X_train, y_train)

# Test the SVM
y_pred = clf.predict(X_test)

# Print the accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test)

misclassified = np.where(y_test != y_pred)

# Print the misclassified labels
print("Misclassified labels: ", y_test[misclassified])

# Print the indices of the misclassified samples
print("Indices of misclassified samples: ", misclassified)